In [1]:
# import fturning

In [6]:
# %load /Users/zhaoxuefeng/Github/zxfMLtools/zxfMLtools/fturning.py
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from peft import (
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

# wget -c


class FTModel():
    def __init__(self):
        self.model = None

    def from_model(self,model, peft_config=None):
        """
        :param model:
        :param peft_config:
        :return:
        """
        config = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型 "CAUSAL_LM"
            inference_mode=False,  # 是否是推理模式
            r=8,  # 中间层神经元的个数
            lora_alpha=32,  # 一个缩放参数
            lora_dropout=0.1  # LoRA层的dropout率
        )
        peft_config = peft_config or config
        model = get_peft_model(model, peft_config)
        print(model.print_trainable_parameters())
        self.model = model

    def train(self,training_args, dataset):
        from transformers import Trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset["train"],  # 训练集
            eval_dataset=dataset["validation"],  # 验证集
            # data_collator=transformers.DataCollatorForSeq2Seq(
            #     tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
            # ),
        )

        # 开始训练和评估模型
        trainer.train()
        # trainer.train#(resume_from_checkpoint=resume_from_checkpoint)
        trainer.evaluate()

    def save(self,output_dir):
        self.model.save_pretrained(output_dir)


import torch.nn as nn
from torchvision import models


class FTModel2():
    def __init__(self):
        self.model = None

    def load(self,model_name,pretrained=True):
        if model_name == 'resnet50':
            model = models.resnet50(pretrained=pretrained)
        return model

    def freeze(self,model):
        # 冻结
        for param in model.parameters():
            param.requires_grad = False

    def replace_network(self,model,class_names=10):
        # 替换最后的全连接层，以适应新的分类任务
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, len(class_names))
        return model


    def from_torchvision(self,model_name, class_names):
        self.model = self.load(model_name=model_name)
        self.freeze(self.model)
        self.model = self.replace_network(self.model)
        return self.model


def func1(self, *_, **__):
  return get_peft_model_state_dict(self, old_state_dict())
model.state_dict = func1.__get__(model, type(model))
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)


   model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model)

model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    if data_path.endswith(".json") or data_path.endswith(".jsonl"):
        data = load_dataset("json", data_files=data_path)
    else:
        data = load_dataset(data_path)

    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")


# trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(),
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

trainer.train(resume_from_checkpoint=resume_from_checkpoint)

model.save_pretrained(output_dir)


train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)

type(data)

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt

    def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result


# %load nlp_weibu/lora_train/lora_train.py
import json
import os.path as osp
from typing import Union
import os
import sys
from typing import List
import fire
import torch
import transformers
from datasets import load_dataset


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        # file_name = osp.join("templates", f"{template_name}.json")
        file_name = "alpaca.json"
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()



"""
Unused imports:
import torch.nn as nn
import bitsandbytes as bnb
"""
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM



def train(
    # model/data params
    base_model: str = "",  # the only required argument
    data_path: str = "yahma/alpaca-cleaned",
    output_dir: str = "./lora-alpaca",
    # training hyperparams
    batch_size: int = 128,
    micro_batch_size: int = 4,
    num_epochs: int = 3,
    learning_rate: float = 3e-4,
    cutoff_len: int = 256,
    val_set_size: int = 2000,
    # lora hyperparams
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    lora_target_modules: List[str] = [
        "q_proj",
        "v_proj",
    ],
    # llm hyperparams
    train_on_inputs: bool = True,  # if False, masks out inputs in loss
    add_eos_token: bool = False,
    group_by_length: bool = False,  # faster, but produces an odd training loss curve
    # wandb params
    wandb_project: str = "",
    wandb_run_name: str = "",
    wandb_watch: str = "",  # options: false | gradients | all
    wandb_log_model: str = "",  # options: false | true
    resume_from_checkpoint: str = None,  # either training checkpoint or final adapter
    prompt_template_name: str = "alpaca",  # The prompt template to use, will default to alpaca.
):
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        print(
            f"Training Alpaca-LoRA model with params:\n"
            f"base_model: {base_model}\n"
            f"data_path: {data_path}\n"
            f"output_dir: {output_dir}\n"
            f"batch_size: {batch_size}\n"
            f"micro_batch_size: {micro_batch_size}\n"
            f"num_epochs: {num_epochs}\n"
            f"learning_rate: {learning_rate}\n"
            f"cutoff_len: {cutoff_len}\n"
            f"val_set_size: {val_set_size}\n"
            f"lora_r: {lora_r}\n"
            f"lora_alpha: {lora_alpha}\n"
            f"lora_dropout: {lora_dropout}\n"
            f"lora_target_modules: {lora_target_modules}\n"
            f"train_on_inputs: {train_on_inputs}\n"
            f"add_eos_token: {add_eos_token}\n"
            f"group_by_length: {group_by_length}\n"
            f"wandb_project: {wandb_project}\n"
            f"wandb_run_name: {wandb_run_name}\n"
            f"wandb_watch: {wandb_watch}\n"
            f"wandb_log_model: {wandb_log_model}\n"
            f"resume_from_checkpoint: {resume_from_checkpoint or False}\n"
            f"prompt template: {prompt_template_name}\n"
        )
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"
    gradient_accumulation_steps = batch_size // micro_batch_size

    prompter = Prompter(prompt_template_name)
    print(prompter,'prompter')
    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps = gradient_accumulation_steps // world_size

    # Check if parameter passed or if set within environ
    use_wandb = len(wandb_project) > 0 or (
        "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
    )
    # Only overwrite environ if wandb param passed
    if len(wandb_project) > 0:
        os.environ["WANDB_PROJECT"] = wandb_project
    if len(wandb_watch) > 0:
        os.environ["WANDB_WATCH"] = wandb_watch
    if len(wandb_log_model) > 0:
        os.environ["WANDB_LOG_MODEL"] = wandb_log_model

    # model = LlamaForCausalLM.from_pretrained(
    #     base_model,
    #     load_in_8bit=True,
    #     torch_dtype=torch.float16,
    #     device_map=device_map,
    # )

    # tokenizer = LlamaTokenizer.from_pretrained(base_model)
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model)


    tokenizer.pad_token_id = (
        0  # unk. we want this to be different from the eos token
    )
    tokenizer.padding_side = "left"  # Allow batched inference

    def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

    def generate_and_tokenize_prompt(data_point):
        full_prompt = prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"],
            data_point["output"],
        )
        tokenized_full_prompt = tokenize(full_prompt)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                data_point["instruction"], data_point["input"]
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    if data_path.endswith(".json") or data_path.endswith(".jsonl"):
        data = load_dataset("json", data_files=data_path)
    else:
        data = load_dataset(data_path)

    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

    if val_set_size > 0:
        train_val = data["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=42
        )
        train_data = (
            train_val["train"].shuffle().map(generate_and_tokenize_prompt)
        )
        val_data = (
            train_val["test"].shuffle().map(generate_and_tokenize_prompt)
        )
    else:
        train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
        val_data = None

    if not ddp and torch.cuda.device_count() > 1:
        # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
        model.is_parallelizable = True
        model.model_parallel = True

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=100,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=200 if val_set_size > 0 else None,
            save_steps=200,
            output_dir=output_dir,
            save_total_limit=3,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )
    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(output_dir)

    print(
        "\n If there's a warning about missing keys above, please disregard :)"
    )
if __name__ == '__main__':
    train(base_model='THUDM/chatglm2-6b',
          data_path='yahma/alpaca-cleaned',
          output_dir='./lora-alpaca',
          batch_size=128,
          )






!pip
install
transformers
!pip
install
peft

# 导入所需的模块和配置类
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 选择一个预训练的模型和分词器
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

# 设置LoRA的配置参数
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型
    inference_mode=False,  # 是否是推理模式
    r=8,  # 中间层神经元的个数
    lora_alpha=32,  # 一个缩放参数
    lora_dropout=0.1  # LoRA层的dropout率
)

# 用预训练的模型初始化一个序列到序列模型类
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# 用get_peft_model函数将LoRA方法应用到模型上
model = get_peft_model(model, peft_config)
print(123)
# 打印出可训练的参数数量和比例
model.print_trainable_parameters()
# output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282

# 用常规的PyTorch或HuggingFace Trainer API来训练模型
# 省略训练代码...


Downloading(…)lve / main / config.json: 0 % | | 0.00 / 800[00:00 <?, ?B / s]
Downloading
pytorch_model.bin: 0 % | | 0.00 / 4.92
G[00:00 <?, ?B / s]
123
trainable
params: 2359296 | | all
params: 1231940608 | | trainable %: 0.19151053100118282
成功

# 导入所需的模块
from datasets import load_dataset
from transformers import AutoTokenizer

# 选择一个分词器
tokenizer_name_or_path = "bigscience/mt0-large"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

# 加载或创建您的数据集
dataset = load_dataset("path/to/your/data")


# 或者
# dataset = Dataset.from_dict(your_data)

# 定义一个分词函数
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])


# 对数据集进行分词
dataset = dataset.map(tokenize_dataset)



!pip
install
datasets

# 导入所需的模块
from datasets import load_dataset
from transformers import AutoTokenizer

# 选择一个分词器
tokenizer_name_or_path = "bigscience/mt0-large"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)


# 定义一个分词函数
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])


Downloading(…)okenizer_config.json: 0 % | | 0.00 / 430[00:00 <?, ?B / s]
Downloading
spiece.model: 0 % | | 0.00 / 4.31
M[00:00 <?, ?B / s]
Downloading
tokenizer.json: 0 % | | 0.00 / 16.3
M[00:00 <?, ?B / s]
Downloading(…)cial_tokens_map.json: 0 % | | 0.00 / 74.0[00:00 <?, ?B / s]

# 导入所需的模块和配置类
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 选择一个预训练的模型和分词器
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

# 设置LoRA的配置参数
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型
    inference_mode=False,  # 是否是推理模式
    r=8,  # 中间层神经元的个数
    lora_alpha=32,  # 一个缩放参数
    lora_dropout=0.1  # LoRA层的dropout率
)

# 用预训练的模型初始化一个序列到序列模型类
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# 用get_peft_model函数将LoRA方法应用到模型上
model = get_peft_model(model, peft_config)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",  # 输出目录
    num_train_epochs=3,  # 训练轮数
    per_device_train_batch_size=16,  # 每个设备上的训练批次大小
    per_device_eval_batch_size=16,  # 每个设备上的评估批次大小
    warmup_steps=500,  # 热身步数
    weight_decay=0.01,  # 权重衰减率
    logging_dir="./logs",  # 日志目录
    logging_steps=10,  # 记录日志的步数间隔
)

# 加载或创建您的数据集，这里我们直接从网上加载一个文本生成数据集[^2^][2]
dataset = load_dataset("totto")


# 定义一个分词函数
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])


# 对数据集进行分词
dataset = dataset.map(tokenize_dataset)

# 创建一个Trainer类实例，并传入模型，训练参数和数据集
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # 训练集
    eval_dataset=dataset["validation"],  # 验证集
)

# 开始训练和评估模型
trainer.train()
trainer.evaluate()

In [ ]:
WARNING: datasets.builder:Found
cached
dataset
totto( / root /.cache / huggingface / datasets / totto / default / 1.0
.0 / 263
c85871e5451bc892c65ca0306c0629eb7beb161e0eb998f56231562335dd2)
0 % | | 0 / 3[00:00 <?, ?it / s]
Map: 0 % | | 0 / 120761[00:00 <?, ? examples / s]
╭─────────────────────────────── Traceback(most
recent
call
last) ────────────────────────────────╮
│ in < cell
line: 46 >:46                                                                            │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / dataset_dict.py: 851 in map                      │
│                                                                                                  │
│    848 │   │   if cache_file_names is None:                                                      │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│ ❱  851 │   │   │   {                                                                             │
│    852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function = function,                                                    │
│    854 │   │   │   │   │   with_indices = with_indices,                                            │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / dataset_dict.py: 852 in < dictcomp >               │
│                                                                                                  │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│    851 │   │   │   {                                                                             │
│ ❱  852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function = function,                                                    │
│    854 │   │   │   │   │   with_indices = with_indices,                                            │
│    855 │   │   │   │   │   with_rank = with_rank,                                                  │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / arrow_dataset.py: 578 in wrapper                 │
│                                                                                                  │
│    575 │   │ else:                                                                             │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    577 │   │  # apply actual function                                                           │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    580 │   │   for dataset in datasets:                                                          │
│    581 │   │   │  # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / arrow_dataset.py: 543 in wrapper                 │
│                                                                                                  │
│    540 │   │   │   "output_all_columns": self._output_all_columns, │
│    541 │   │}                                                                                 │
│    542 │   │  # apply actual function                                                           │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    544 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    545 │   │  # re-apply format to the output                                                   │
│    546 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / arrow_dataset.py: 3073 in map                    │
│                                                                                                  │
│   3070 │   │   │   │   │   leave = False,                                                          │
│   3071 │   │   │   │   │   desc = desc or "Map",                                                   │
│   3072 │   │   │   │   ) as pbar:                                                                │
│ ❱ 3073 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwargs):     │
│   3074 │   │   │   │   │   │   if done:                                                          │
│   3075 │   │   │   │   │   │   │   shards_done += 1                                              │
│   3076 │   │   │   │   │   │   │   logger.debug(f"Finished processing shard number {rank} of {n  │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / arrow_dataset.py: 3427 in _map_single            │
│                                                                                                  │
│   3424 │   │   │   │   if not batched:                                                           │
│   3425 │   │   │   │   │   _time = time.time()                                                   │
│   3426 │   │   │   │   │   for i, example in shard_iterable:                                     │
│ ❱ 3427 │   │   │   │   │   │   example = apply_function_on_filtered_inputs(example, i, offset=o  │
│   3428 │   │   │   │   │   │   if update_data:                                                   │
│   3429 │   │   │   │   │   │   │   if i == 0:                                                    │
│   3430 │   │   │   │   │   │   │   │   buf_writer, writer, tmp_file = init_buffer_and_writer()   │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / arrow_dataset.py: 3330 in                        │
│ apply_function_on_filtered_inputs                                                                │
│                                                                                                  │
│   3327 │   │   │   │   additional_args += (effective_indices,)                                   │
│   3328 │   │   │   if with_rank:                                                                 │
│   3329 │   │   │   │   additional_args += (rank,)                                                │
│ ❱ 3330 │   │   │   processed_inputs = function(*fn_args, *additional_args, **fn_kwargs)          │
│   3331 │   │   │   if isinstance(processed_inputs, LazyDict):                                    │
│   3332 │   │   │   │   processed_inputs = {                                                      │
│   3333 │   │   │   │   │   k: v
for k, v in processed_inputs.data.items() if k not in processed  │
│ in tokenize_dataset:43                                                                           │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / datasets / formatting / formatting.py: 270 in __getitem__     │
│                                                                                                  │
│   267 │   │   return len(self.data)                                                              │
│   268 │                                                                                          │
│   269 │

def __getitem__(self, key):                                                            │

│ ❱ 270 │   │   value = self.data[key]                                                             │
│   271 │   │   if key in self.keys_to_format:                                                     │
│   272 │   │   │   value = self.format(key)                                                       │
│   273 │   │   │   self.data[key] = value                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'text'

# final

# 导入所需的模块和配置类
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 选择一个预训练的模型和分词器
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

# 设置LoRA的配置参数
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型
    inference_mode=False,  # 是否是推理模式
    r=8,  # 中间层神经元的个数
    lora_alpha=32,  # 一个缩放参数
    lora_dropout=0.1  # LoRA层的dropout率
)

# 用预训练的模型初始化一个序列到序列模型类
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# 用get_peft_model函数将LoRA方法应用到模型上
model = get_peft_model(model, peft_config)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",  # 输出目录
    num_train_epochs=3,  # 训练轮数
    per_device_train_batch_size=16,  # 每个设备上的训练批次大小
    per_device_eval_batch_size=16,  # 每个设备上的评估批次大小
    warmup_steps=500,  # 热身步数
    weight_decay=0.01,  # 权重衰减率
    logging_dir="./logs",  # 日志目录
    logging_steps=10,  # 记录日志的步数间隔
)

# 加载或创建您的数据集，这里我们直接从网上加载一个文本生成数据集
dataset = load_dataset("totto")

WARNING: datasets.builder:Found
cached
dataset
totto( / root /.cache / huggingface / datasets / totto / default / 1.0
.0 / 263
c85871e5451bc892c65ca0306c0629eb7beb161e0eb998f56231562335dd2)
0 % | | 0 / 3[00:00 <?, ?it / s]

dataset.keys()

dict_keys(['train', 'validation', 'test'])

dataset['train']

Dataset({
    features: ['id', 'table_page_title', 'table_webpage_url', 'table_section_title', 'table_section_text', 'table',
               'highlighted_cells', 'example_id', 'sentence_annotations', 'overlap_subset'],
    num_rows: 120761
})


# 定义一个分词函数，注意修改键名为"sentence_annotations"，以匹配数据集的格式
# 并且从字典中选择"final_sentence"作为文本数据
# def tokenize_dataset(data):
#     # print(data,'data')
#     # Keys of the returned dictionary will be added to the dataset as columns
#     return tokenizer(data["sentence_annotations"]["final_sentence"])

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence_annotations"]["final_sentence"], truncation=True)


# 创建一个分词器对象
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

# 定义一个分词函数，注意修改键名为"final_sentence"，以匹配数据集的格式
# def tokenize_dataset(data):
#     # Keys of the returned dictionary will be added to the dataset as columns
#     return tokenizer(data["sentence_annotations"])

# 对数据集进行分词
dataset = dataset.map(tokenize_dataset)

# 创建一个Trainer类实例，并传入模型，训练参数和数据集
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # 训练集
    eval_dataset=dataset["validation"],  # 验证集
)

# 开始训练和评估模型
trainer.train()
trainer.evaluate()

Map: 0 % | | 0 / 120761[00:00 <?, ? examples / s]
Asking
to
truncate
to
max_length
but
no
maximum
length is provided and the
model
has
no
predefined
maximum
length.Default
to
no
truncation.
Map: 0 % | | 0 / 7700[00:00 <?, ? examples / s]
Map: 0 % | | 0 / 7700[00:00 <?, ? examples / s]
╭─────────────────────────────── Traceback(most
recent
call
last) ────────────────────────────────╮
│ in < cell
line: 21 >:21                                                                            │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / transformers / trainer.py: 1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args = args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint = resume_from_checkpoint,                                │
│   1667 │   │   │   trial = trial,                                                                  │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / transformers / trainer.py: 1909 in _inner_training_loop     │
│                                                                                                  │
│   1906 │   │   │   │   rng_to_sync = True                                                        │
│   1907 │   │   │                                                                                 │
│   1908 │   │   │   step = -1                                                                     │
│ ❱ 1909 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1910 │   │   │   │   total_batched_samples += 1                                                │
│   1911 │   │   │   │   if rng_to_sync:                                                           │
│   1912 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / torch / utils / data / dataloader.py: 634 in __next__           │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │  # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    637 │   │   │   │   │   self._IterableDataset_len_called is not None and  \                    │
│                                                                                                  │
│ / usr / local / lib / python3.10 / dist-packages / torch / utils / data / dataloader.py:678 in _next_data         │
│                                                                                                  │
│    675 │                                                                                         │
│    676 │

def _next_data(self):                                                                 │

│    677 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  678 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    679 │   │   if self._pin_memory:                                                              │
│    680 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    681 │   │   return data                                                                       │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / torch / utils / data / _utils / fetch.py: 54 in fetch             │
│                                                                                                  │
│   51 │   │   │   │   data = [self.dataset[idx] for idx in possibly_batched_index]                │
│   52 │   │ else:                                                                               │
│   53 │   │   │   data = self.dataset[possibly_batched_index]                                     │
│ ❱ 54 │   │   return self.collate_fn(data)                                                        │
│   55                                                                                             │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / transformers / data / data_collator.py: 70 in                 │
│ default_data_collator                                                                            │
│                                                                                                  │
│     67 │  # on the whole batch.                                                                 │
│     68 │                                                                                         │
│     69 │   if return_tensors == "pt":                                                            │
│ ❱   70 │   │   return torch_default_data_collator(features)                                      │
│     71 │ elif return_tensors == "tf":                                                          │
│     72 │   │   return tf_default_data_collator(features)                                         │
│     73 │ elif return_tensors == "np":                                                          │
│                                                                                                  │
│ / usr / local / lib / python3
.10 / dist - packages / transformers / data / data_collator.py: 136 in                │
│ torch_default_data_collator                                                                      │
│                                                                                                  │
│    133 │   │   │ elif isinstance(v, np.ndarray):                                               │
│    134 │   │   │   │   batch[k] = torch.tensor(np.stack([f[k] for f in features]))               │
│    135 │   │   │ else:                                                                         │
│ ❱  136 │   │   │   │   batch[k] = torch.tensor([f[k] for f in features])                         │
│    137 │                                                                                         │
│    138 │   return batch                                                                          │
│    139                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: expected
sequence
of
length
25
at
dim
2(got
28)




# final

# 导入所需的模块和配置类
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 选择一个预训练的模型和分词器
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

# 设置LoRA的配置参数
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型
    inference_mode=False,  # 是否是推理模式
    r=8,  # 中间层神经元的个数
    lora_alpha=32,  # 一个缩放参数
    lora_dropout=0.1  # LoRA层的dropout率
)

# 用预训练的模型初始化一个序列到序列模型类
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# 用get_peft_model函数将LoRA方法应用到模型上
model = get_peft_model(model, peft_config)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",  # 输出目录
    num_train_epochs=3,  # 训练轮数
    per_device_train_batch_size=16,  # 每个设备上的训练批次大小
    per_device_eval_batch_size=16,  # 每个设备上的评估批次大小
    warmup_steps=500,  # 热身步数
    weight_decay=0.01,  # 权重衰减率
    logging_dir="./logs",  # 日志目录
    logging_steps=10,  # 记录日志的步数间隔
)

# 加载或创建您的数据集，这里我们直接从网上加载一个文本生成数据集
dataset = load_dataset("totto")


def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence_annotations"]["final_sentence"], truncation=True)


# 创建一个分词器对象
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

# 定义一个分词函数，注意修改键名为"final_sentence"，以匹配数据集的格式
# def tokenize_dataset(data):
#     # Keys of the returned dictionary will be added to the dataset as columns
#     return tokenizer(data["sentence_annotations"])

# 对数据集进行分词
dataset = dataset.map(tokenize_dataset)

# 创建一个Trainer类实例，并传入模型，训练参数和数据集
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # 训练集
    eval_dataset=dataset["validation"],  # 验证集
)

# 开始训练和评估模型
trainer.train()
trainer.evaluate()

ValueError: expected
sequence
of
length
25
at
dim
2(got
28)





# 导入所需的模块和配置类
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

# 选择一个预训练的模型和分词器
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

# 设置LoRA的配置参数
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # 任务类型
    inference_mode=False,  # 是否是推理模式
    r=8,  # 中间层神经元的个数
    lora_alpha=32,  # 一个缩放参数
    lora_dropout=0.1  # LoRA层的dropout率
)

# 用预训练的模型初始化一个序列到序列模型类
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

# 用get_peft_model函数将LoRA方法应用到模型上
model = get_peft_model(model, peft_config)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",  # 输出目录
    num_train_epochs=3,  # 训练轮数
    per_device_train_batch_size=16,  # 每个设备上的训练批次大小
    per_device_eval_batch_size=16,  # 每个设备上的评估批次大小
    warmup_steps=500,  # 热身步数
    weight_decay=0.01,  # 权重衰减率
    logging_dir="./logs",  # 日志目录
    logging_steps=10,  # 记录日志的步数间隔
)

# 加载或创建您的数据集，这里我们直接从网上加载一个文本生成数据集
dataset = load_dataset("totto")


# 定义一个分词函数，注意修改键名为"sentence_annotations"，以匹配数据集的格式
# 并且从字典中选择"final_sentence"作为文本数据
# 并且添加一个参数，将文本截断到模型的最大长度

# def tokenize_dataset(data):
#     # Keys of the returned dictionary will be added to the dataset as columns
#     return tokenizer(data["sentence_annotations"]["final_sentence"], truncation=True)

def tokenize_dataset(data, tokenizer):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence_annotations"]["final_sentence"], truncation=True)


# 创建一个分词器对象，并传入分词函数中作为参数
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
dataset = dataset.map(lambda x: tokenize_dataset(x, tokenizer))

# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
# dataset = dataset.map(lambda x: tokenize_dataset(x, tokenizer))

# 创建一个Trainer类实例，并传入模型，训练参数和数据集
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # 训练集
    eval_dataset=dataset["validation"],  # 验证集
)

# 开始训练和评估模型
trainer.train()
trainer.evaluate()